# FIT5196 Assessment 1
#### Student Name:Aishwarya Srirangam Murali 
#### Student ID: 29999715

Date: 16/09/2020

Libraries used:
* re(for regular expression, included in Anaconda Python 3.6) 
* os (For interactig with the files and operation systems)
* langid (for language detection and processing)
* pandas 0.19.2 (for data frame, included in Anaconda Python 3.6) 
* nltk 3.2.2 (Natural Language Toolkit, included in Anaconda Python 3.6)
* nltk.collocations (for finding bigrams, included in Anaconda Python 3.6)
* nltk.tokenize (for tokenization, included in Anaconda Python 3.6)
* nltk.corpus (for stop words, not included in Anaconda, `nltk.download('stopwords')` provided)
* nltk.stem (for stemming)
* nltk.util (for ngrams)
* numpy (For arrays and matrices)

## 1. Introduction
This assignment comprises the extraction of data from excel file. We are given excel file which contains tweets where we are to extract the tweet and the text associated with the tweet. After we have extracted the tweet we are to filter the english tweets. Next we have to generate the unigrams, bigrams and the sample count. 

More details for each task will be given in the following sections.

## 2.  Importing all the libraries 


In [1]:
import re
import pandas as pd
import nltk
import numpy as np
import langid
from tqdm import tqdm_notebook
from collections import Counter, OrderedDict
from itertools import chain
from nltk.tokenize import RegexpTokenizer,  MWETokenizer
from nltk.stem import PorterStemmer
from nltk.util import ngrams

## 2.  Importing all the libraries 
* Here, we are first reading the excel file 
* Dropping all the empty rows and columns
* Extracting the text, id and created at 
* Concatenate all of it 

In [2]:
#Reading the excel file 
data = pd.read_excel('part2/Excel file part2.xlsx', sheet_name=None)

dfs = []

for date in data.keys():
    df =  data[date].copy()
    #Dropping all the rows and columns with null values 
    df.dropna(axis=1, how='all', inplace=True) #drop empty cols 
    df.dropna(axis=0, how='all', inplace=True) #drop empty rows
    df.reset_index(inplace=True, drop=True)
    if ('text' in df.columns) and ('id' in df.columns) and ('created_at' in df.columns):
        pass
    else:
        df.columns = df.iloc[0, :]
        df = df.iloc[1:]
    assert ('text' in df.columns) and ('id' in df.columns) and ('created_at' in df.columns) and (len(df.columns)==3)
    df['date'] = date
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
df.head()

,text,id,created_at,date
0,"Yes, all networks need to stop covering Trump ...",1241579835371278080,2020-03-22T04:17:54.000Z,2020-03-22
1,Oga Sir! Directive for closure of all public g...,1241624080391994880,2020-03-22T07:13:43.000Z,2020-03-22
2,Stay at home so the js virus doesn't spread\n\...,1241680036803087872,2020-03-22T10:56:04.000Z,2020-03-22
3,Médicos y personal sanitario infectado: 3.475 ...,1241713141941240064,2020-03-22T13:07:37.000Z,2020-03-22
4,Chinese scientists dismiss claims Covid19 pati...,1241605146758569984,2020-03-22T05:58:29.000Z,2020-03-22


In [3]:
print(len(df))

162000


In [4]:
#Writing a function for langid - classifying the language 
def classify_lang(x):
    try:
        return langid.classify(x)[0]
    except:
        return '-1'

In [5]:
df['lang'] = [classify_lang(x) for x in tqdm_notebook(df['text'])]
df.head()

,text,id,created_at,date,lang
0,"Yes, all networks need to stop covering Trump ...",1241579835371278080,2020-03-22T04:17:54.000Z,2020-03-22,en
1,Oga Sir! Directive for closure of all public g...,1241624080391994880,2020-03-22T07:13:43.000Z,2020-03-22,en
2,Stay at home so the js virus doesn't spread\n\...,1241680036803087872,2020-03-22T10:56:04.000Z,2020-03-22,en
3,Médicos y personal sanitario infectado: 3.475 ...,1241713141941240064,2020-03-22T13:07:37.000Z,2020-03-22,es
4,Chinese scientists dismiss claims Covid19 pati...,1241605146758569984,2020-03-22T05:58:29.000Z,2020-03-22,en


## 3. Filtering non english tweets 
* Dropping all the tweets that are not english

In [6]:
#Keep english tweets only and dropping the others
print(df.shape)
df = df[df['lang'] == 'en']
print(df.shape)
df.head()

(162000, 5)
(91733, 5)


,text,id,created_at,date,lang
0,"Yes, all networks need to stop covering Trump ...",1241579835371278080,2020-03-22T04:17:54.000Z,2020-03-22,en
1,Oga Sir! Directive for closure of all public g...,1241624080391994880,2020-03-22T07:13:43.000Z,2020-03-22,en
2,Stay at home so the js virus doesn't spread\n\...,1241680036803087872,2020-03-22T10:56:04.000Z,2020-03-22,en
4,Chinese scientists dismiss claims Covid19 pati...,1241605146758569984,2020-03-22T05:58:29.000Z,2020-03-22,en
6,We want to make this a special mother’s day! A...,1241666046681140992,2020-03-22T10:00:29.000Z,2020-03-22,en


## 4. Concatenating and defining functions 
* First, concatenating all the tweets 
* We are using the regular expression to tokenize 
* Adding the top 200 bigrams to the vocabulary 
* Stemming using porter stemmer
* Removing the stop words 
* Dropping the words which are not in the vocabulary 
* Writing it to the file 
* Reading the file 


In [7]:
#Concatenating all the tweets for the date 
raw_data = {}
for date in df['date'].unique():
    raw_data[date] = " ".join(df[df['date']==date]['text'])

In [8]:
#Using the regular expression as given to get the tokenised data 
def tokenize_data(data):
    tokenizer = RegexpTokenizer(r'[a-zA-Z]{2,}')
    #converting everything to lower case 
    tokenized_data = dict((date, tokenizer.tokenize(text.lower())) for date, text in data.items())
    return tokenized_data


#Top 200 bigrams to the vocabulary
def add_collocations(data):
    all_words = list(chain.from_iterable(data.values()))
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(all_words)
    bigram_finder.apply_freq_filter(20)
    bigram_finder.apply_word_filter(lambda w: len(w) < 3)
    top_200_bigrams = bigram_finder.nbest(bigram_measures.pmi, 200)
    mwetokenizer = MWETokenizer(top_200_bigrams)
    colloc_data =  dict((date, mwetokenizer.tokenize(tokens)) for date, tokens in data.items())
    return colloc_data


#Stemming using Porter stemmer 
def stem_data(data):
    stemmer = PorterStemmer()
    stemmed_data = dict((date, np.vectorize(stemmer.stem)(tokens)) for date, tokens in data.items())
    return stemmed_data


#Removing the stop words and removing the tokens where the length is less than 3 
def make_vocab(data, stop_words=[]):
    unique_tokens = np.concatenate([np.unique(tokens) for date, tokens in data.items()]).flatten()
    counter = Counter(unique_tokens)
    vocab = list(counter)
    vocab = [token for token in vocab if counter[token]>5] #Removing rare tokens
    vocab = [token for token in vocab if counter[token]<60] #context-dependent stop tokens
    vocab = [token for token in vocab if token not in stop_words] #context-independent stop tokens
    vocab = [token for token in vocab if len(token)>3] 
    return OrderedDict((token, i) for token, i in zip(sorted(vocab), range(len(vocab))))


#Dropping the words which are not in vocabulary 
def filter_based_on_vocab(data, vocab):
    filtered_data = {}
    for date, tokens in data.items():
        filtered_data[date] = list(filter(lambda x: x in vocab, tokens))
    return filtered_data


#Writing it to the file 
def write_to_file(data, filepath, join_using=':'):
    with open(filepath, 'w') as f:
        for k, v in tqdm_notebook(data.items()):
            f.write(f'{k}{join_using}{str(v)}\n')
            
            
#Reading the file     
def read_stop_words(filepath):
    return list(np.loadtxt(filepath, dtype=str))

## 5. Functions
* Calling all the functions defined above 
* Writing a functions for n grams 
* Taking the first 100 for both unigram and bigram 
* Writing it to the files 

In [36]:
#Calling all of the functions defined above 
data = raw_data.copy()
data = tokenize_data(data)
data = add_collocations(data)
data = stem_data(data)
stop_words = read_stop_words('part2/stopwords_en.txt')
vocab = make_vocab(data, stop_words=stop_words) 
data = filter_based_on_vocab(data, vocab)


write_to_file(vocab, 'part2/29999715_vocab.txt')

In [23]:
#Writing a function for n grams 
def top_k_freq_ngrams(tokens:list, n:int, k:int,):
    all_n_grams = ngrams(sequence=tokens, n=n)
    freq = Counter(all_n_grams)
    k_most_common = freq.most_common(k)
    if n == 1:
        k_most_common = [(token[0], count) for token, count in k_most_common]
    return k_most_common

In [24]:
#Calling the function written above and taking the first 100 for both unigram and bigram 
unigrams = dict((date, top_k_freq_ngrams(tokens, n=1, k=100)) for date, tokens in data.items())
bigrams = dict((date, top_k_freq_ngrams(tokens, n=2, k=100)) for date, tokens in data.items())

In [37]:
#Key of the dict, string of the value and 
#saving the unigram and bigram file 
write_to_file(unigrams, 'part2/29999715_100uni.txt')
write_to_file(bigrams, 'part2/29999715_100bi.txt')

In [27]:
#List of the documents from string of words 
corpus = [" ".join(tokens) for date, tokens in data.items()]
#The dates 
docids = [date for date, tokens in data.items()]

In [28]:
#Sparse matrix 
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [35]:
countvec_dict = dict((date, ",".join(list(map(str, vec)))) for date, vec in zip(docids, X.toarray()))
write_to_file(countvec_dict, './part2/29999715_countVec.txt', join_using=',')

In [31]:
assert len(vectorizer.get_feature_names()) == len(vocab)

## 6. Reference
1. re — Regular expression operations — Python 3.8.6rc1 documentation. (2020). Retrieved 16 September 2020, from https://docs.python.org/3/library/re.html
2. os — Miscellaneous operating system interfaces — Python 3.8.6rc1 documentation. (2020). Retrieved 16 September 2020, from https://docs.python.org/3/library/os.html
3. langid. (2020). Retrieved 16 September 2020, from https://pypi.org/project/langid/1.1dev/
4. xmltodict. (2020). Retrieved 16 September 2020, from https://pypi.org/project/xmltodict/
5. pandas - Python Data Analysis Library. (2020). Retrieved 16 September 2020, from https://pandas.pydata.org/
6. nltk. (2020). Retrieved 16 September 2020, from https://pypi.org/project/nltk/
7. NumPy. (2020). Retrieved 16 September 2020, from https://numpy.org/    